In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
def image_masking(filepath):
	
	BLUR = 21
	CANNY_THRESH_1 = 10
	CANNY_THRESH_2 = 200
	MASK_DILATE_ITER = 10
	MASK_ERODE_ITER = 10
	MASK_COLOR = (0.0,0.0,0.0) # In BGR format
	
	img = cv2.imread(filepath)
	gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
	
	edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
	edges = cv2.dilate(edges, None)
	edges = cv2.erode(edges, None)
	
	contour_info = []
	contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
	
	for c in contours:
	    contour_info.append((c, cv2.isContourConvex(c), cv2.contourArea(c),))
	contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
	
	max_contour = contour_info[0]
	mask = np.zeros(edges.shape)
	cv2.fillConvexPoly(mask, max_contour[0], (255))
	
	mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
	mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
	mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
	
	mask_stack = np.dstack([mask]*3)
	mask_stack  = mask_stack.astype('float32') / 255.0
	img = img.astype('float32') / 255.0
	
	masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR)
	masked = (masked * 255).astype('uint8')

	fileName, fileExtension = filepath.split('.')
	fileName += '-masked.'
	filepath = fileName + fileExtension
	print(filepath)

	cv2.imwrite(filepath, masked)

In [3]:
img = 'blackmeasles (1).jpg'
image_masking(img)

blackmeasles (1)-masked.jpg


In [4]:
def ProcessImage1(im):
    resized = cv2.resize(im, (500,500))
    cv2.imshow("Resized", resized)
    cv2.waitKey(0)
    blurred = cv2.medianBlur(resized, 3)
    #image = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    cv2.imshow("MedBlur", blurred)
    cv2.waitKey(0)
    return(blurred)

In [27]:
def segment(image, threshold=25):
    global bg
    bg = image.copy()
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)
    cv2.imshow("diff", diff)
    cv2.waitKey(0)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    img = thresholded

    # get the contours in the thresholded image
    # _, cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    _, cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
            # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
    cv2.imshow("thresholded", thresholded)
    cv2.waitKey(0)
        
    cv2.imshow("Segment", segmented)
    cv2.waitKey(0)
    return (thresholded, segmented)


In [28]:
img = cv2.imread('blackmeasles (1).jpg')

#gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img1 = ProcessImage1(img)

segment(img1)


error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\thresh.cpp:1557: error: (-2:Unspecified error) in function 'double __cdecl cv::threshold(const class cv::_InputArray &,const class cv::_OutputArray &,double,double,int)'
> THRESH_OTSU mode:
>     'src_type == CV_8UC1 || src_type == CV_16UC1'
> where
>     'src_type' is 16 (CV_8UC3)
